# Weather forecast (OpenSource)

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## IMPORTS

In [2]:
# ALL
import json
import ast
import re
import pickle
import time, datetime

# AS
import numpy as np, pandas as pd

# FROM
# from dateutil.relativedelta import relativedelta

# FROM .PY FILES
from convert_columns_to_floats import *

## LOAD DATA

In [3]:
weather_modality_directory = "/Users/izzymohamed/Desktop/Fay/Data/forecasts"

In [4]:
# load data
df_for = pd.read_csv(weather_modality_directory+'/open_weather.csv', index_col='timestamp', parse_dates=True, date_parser=pd.to_datetime)

In [5]:
df_for.head()

,temperature,humidity,pressure,cloud_cover,wind_speed,wind_direction,weather_code,poprecipitation
timestamp,,,,,,,,
2021-04-30 22:00:00,"[15.56, 15.26, 14.84, 14.2, 13.18, 12.13, 12.0...","[67, 69, 71, 74, 79, 84, 85, 81, 73, 66, 61, 5...","[1010, 1010, 1010, 1010, 1010, 1010, 1010, 101...","[0, 18, 34, 60, 57, 75, 81, 85, 86, 100, 100, ...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252, 242, 243, 221, 230, 223, 231, 229, 221, ...","{'id': [800, 801, 802, 803, 803, 803, 803, 804...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02, ..."
2021-04-30 22:15:00,"[15.56, 15.26, 14.84, 14.2, 13.18, 12.13, 12.0...","[67, 69, 71, 74, 79, 84, 85, 81, 73, 66, 61, 5...","[1010, 1010, 1010, 1010, 1010, 1010, 1010, 101...","[0, 18, 34, 60, 57, 75, 81, 85, 86, 100, 100, ...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252, 242, 243, 221, 230, 223, 231, 229, 221, ...","{'id': [800, 801, 802, 803, 803, 803, 803, 804...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02, ..."
2021-04-30 22:30:00,"[15.36, 15.56, 15.2, 14.65, 13.78, 12.82, 12.0...","[69, 67, 69, 72, 76, 81, 85, 81, 73, 66, 61, 5...","[1010, 1010, 1010, 1010, 1010, 1010, 1010, 101...","[18, 0, 17, 40, 43, 60, 81, 85, 86, 100, 100, ...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252, 242, 243, 221, 230, 223, 231, 229, 221, ...","{'id': [801, 800, 801, 802, 802, 803, 803, 804...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02, ..."
2021-04-30 22:45:00,"[15.28, 15.47, 15.13, 14.6, 13.74, 12.8, 12.08...","[73, 72, 73, 75, 78, 82, 85, 81, 73, 66, 61, 5...","[1010, 1010, 1010, 1010, 1010, 1010, 1010, 101...","[18, 0, 17, 40, 43, 60, 81, 85, 86, 100, 100, ...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252, 242, 243, 221, 230, 223, 231, 229, 221, ...","{'id': [801, 800, 801, 802, 802, 803, 803, 804...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02, ..."
2021-04-30 23:00:00,"[15.48, 15.13, 14.6, 13.75, 12.8, 12.08, 13.32...","[72, 73, 75, 78, 82, 85, 81, 73, 66, 61, 56, 5...","[1010, 1010, 1010, 1010, 1010, 1010, 1011, 101...","[0, 17, 40, 43, 60, 81, 85, 86, 100, 100, 100,...","[1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84, 0.54,...","[242, 243, 221, 230, 223, 231, 229, 221, 239, ...","{'id': [800, 801, 802, 802, 803, 803, 804, 804...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02, 0.2..."


In [6]:
df_for.columns

Index(['temperature', 'humidity', 'pressure', 'cloud_cover', 'wind_speed',
       'wind_direction', 'weather_code', 'poprecipitation'],
      dtype='object')

## CONVERT STRING TO FLOAT

In [7]:
df_for = convert_columns_to_floats2(df_for)

Processing column: temperature
Processed column: temperature
Processing column: humidity
Processed column: humidity
Processing column: pressure
Processed column: pressure
Processing column: cloud_cover
Processed column: cloud_cover
Processing column: wind_speed
Processed column: wind_speed
Processing column: wind_direction
Processed column: wind_direction
Processing column: weather_code
Processed column: weather_code
Processing column: poprecipitation
Processed column: poprecipitation


In [8]:
df_for.head()

,temperature,humidity,pressure,cloud_cover,wind_speed,wind_direction,weather_code,poprecipitation
timestamp,,,,,,,,
2021-04-30 22:00:00,"[15.56, 15.26, 14.84, 14.2, 13.18, 12.13, 12.0...","[67.0, 69.0, 71.0, 74.0, 79.0, 84.0, 85.0, 81....","[1010.0, 1010.0, 1010.0, 1010.0, 1010.0, 1010....","[0.0, 18.0, 34.0, 60.0, 57.0, 75.0, 81.0, 85.0...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252.0, 242.0, 243.0, 221.0, 230.0, 223.0, 231...","[800.0, 801.0, 802.0, 803.0, 803.0, 803.0, 803...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-04-30 22:15:00,"[15.56, 15.26, 14.84, 14.2, 13.18, 12.13, 12.0...","[67.0, 69.0, 71.0, 74.0, 79.0, 84.0, 85.0, 81....","[1010.0, 1010.0, 1010.0, 1010.0, 1010.0, 1010....","[0.0, 18.0, 34.0, 60.0, 57.0, 75.0, 81.0, 85.0...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252.0, 242.0, 243.0, 221.0, 230.0, 223.0, 231...","[800.0, 801.0, 802.0, 803.0, 803.0, 803.0, 803...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-04-30 22:30:00,"[15.36, 15.56, 15.2, 14.65, 13.78, 12.82, 12.0...","[69.0, 67.0, 69.0, 72.0, 76.0, 81.0, 85.0, 81....","[1010.0, 1010.0, 1010.0, 1010.0, 1010.0, 1010....","[18.0, 0.0, 17.0, 40.0, 43.0, 60.0, 81.0, 85.0...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252.0, 242.0, 243.0, 221.0, 230.0, 223.0, 231...","[801.0, 800.0, 801.0, 802.0, 802.0, 803.0, 803...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-04-30 22:45:00,"[15.28, 15.47, 15.13, 14.6, 13.74, 12.8, 12.08...","[73.0, 72.0, 73.0, 75.0, 78.0, 82.0, 85.0, 81....","[1010.0, 1010.0, 1010.0, 1010.0, 1010.0, 1010....","[18.0, 0.0, 17.0, 40.0, 43.0, 60.0, 81.0, 85.0...","[1.72, 1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84,...","[252.0, 242.0, 243.0, 221.0, 230.0, 223.0, 231...","[801.0, 800.0, 801.0, 802.0, 802.0, 803.0, 803...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-04-30 23:00:00,"[15.48, 15.13, 14.6, 13.75, 12.8, 12.08, 13.32...","[72.0, 73.0, 75.0, 78.0, 82.0, 85.0, 81.0, 73....","[1010.0, 1010.0, 1010.0, 1010.0, 1010.0, 1010....","[0.0, 17.0, 40.0, 43.0, 60.0, 81.0, 85.0, 86.0...","[1.5, 1.94, 1.8, 1.66, 1.28, 1.15, 0.84, 0.54,...","[242.0, 243.0, 221.0, 230.0, 223.0, 231.0, 229...","[800.0, 801.0, 802.0, 802.0, 803.0, 803.0, 804...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
len(df_for['temperature'])

63132

In [10]:
for column in df_for.columns:
    print(column)
    print(get_length_of_list(df_for, column))
    print("\n")

temperature
timestamp
2021-04-30 22:00:00    48
2021-04-30 22:15:00    48
2021-04-30 22:30:00    48
2021-04-30 22:45:00    48
2021-04-30 23:00:00    48
                       ..
2023-02-17 11:45:00    48
2023-02-17 12:00:00    48
2023-02-17 12:15:00    48
2023-02-17 12:30:00    48
2023-02-17 12:45:00    48
Name: temperature, Length: 63132, dtype: int64


humidity
timestamp
2021-04-30 22:00:00    48
2021-04-30 22:15:00    48
2021-04-30 22:30:00    48
2021-04-30 22:45:00    48
2021-04-30 23:00:00    48
                       ..
2023-02-17 11:45:00    48
2023-02-17 12:00:00    48
2023-02-17 12:15:00    48
2023-02-17 12:30:00    48
2023-02-17 12:45:00    48
Name: humidity, Length: 63132, dtype: int64


pressure
timestamp
2021-04-30 22:00:00    48
2021-04-30 22:15:00    48
2021-04-30 22:30:00    48
2021-04-30 22:45:00    48
2021-04-30 23:00:00    48
                       ..
2023-02-17 11:45:00    48
2023-02-17 12:00:00    48
2023-02-17 12:15:00    48
2023-02-17 12:30:00    48
2023-02-17 12

In [11]:
for column in df_for.columns:
    print(column, get_length_of_list(df_for, column).unique())

temperature [48  0]
humidity [48  0]
pressure [48  0]
cloud_cover [48  0]
wind_speed [48  0]
wind_direction [48  0]
weather_code [96  0]
poprecipitation [48  0]


## REDUCE TO 3 DAYS

In [12]:
df_for_reduced = reduce_cells_for_all_columns(df_for, 42)

In [13]:
for column in df_for.columns:
    print(column, get_length_of_list(df_for_reduced, column).unique())

temperature [6 0]
humidity [6 0]
pressure [6 0]
cloud_cover [6 0]
wind_speed [6 0]
wind_direction [6 0]
weather_code [54  0]
poprecipitation [6 0]


## Memory Reducer

In [14]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df